# Import statments

In [ ]:

import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array

# main imports for model fitting 
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D,Input,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models,optimizers
from tensorflow.keras.regularizers import l2

#model tuning imports
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.optimizers import Adam, RMSprop

from tensorflow.keras.models import Model

#evaluation improts
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report




# Preprocessing and loading data

In [ ]:
# This is a function i made to help read the training images in RGB format
def load_images_from_folder(folder, image_ids):
    images = []
    #loops through each image file in the given folder (train or test)
    for img_id in image_ids:
        img_path = os.path.join(folder, f"image_{img_id}.png") # concatinating
        image = cv2.imread(img_path)
        if image is not None:
            # cv2 automatically uses BGR format so changed it to RGB for the model to learn better
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
            image = cv2.resize(image, (32, 32)) #ensuring image size is 32*32
            images.append(image)
        else:
            print(f"Image {img_path} could not be loaded.")
    return np.array(images)

# Load training data
train_df = pd.read_csv('/kaggle/input/complete-data-set/train.csv')

# Load images and labels
train_images = load_images_from_folder('/kaggle/input/complete-data-set/cifar10_images/train', train_df['id'].values)
train_labels = train_df['label'].values.astype(int)  # Ensure labels are integers

# Normalize images to help model learn better
train_images = train_images / 255.0

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.3, random_state=42)


In [ ]:
# Test data preprocessing
#Using split function to get test image id's from the filenames of each image as that's the format they came in
test_ids = [f.split('.')[0].split('_')[1] for f in os.listdir('/kaggle/input/complete-data-set/cifar10_images/test')]
test_images = load_images_from_folder('/kaggle/input/complete-data-set/cifar10_images/test', test_ids)
# Normalize images to match the training data
test_images = test_images / 255.0


In [ ]:
# Save preprocessed data as numpy arrays so that we don't have to rerun the steps everytime 
np.save('/kaggle/working/X_train.npy', X_train)
np.save('/kaggle/working/y_train.npy', y_train)
np.save('/kaggle/working/X_val.npy', X_val)
np.save('/kaggle/working/y_val.npy', y_val)

np.save('/kaggle/working/test_images.npy', test_images)
np.save('/kaggle/working/test_ids.npy', test_ids)



In [ ]:
#load required data everytime we restart the notebook 

X_train = np.load('/kaggle/working/X_train.npy')
y_train = np.load('/kaggle/working/y_train.npy')
X_val = np.load('/kaggle/working/X_val.npy')                  
y_val = np.load('/kaggle/working/y_val.npy')
test_images = np.load('/kaggle/working/test_images.npy')
test_ids = np.load('/kaggle/working/test_ids.npy')

In [ ]:
#checking the shape of our data to insure it's correct before we feed it into the model
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, test_images.shape, test_ids.shape)

# Visualisation

In [ ]:
#checking some of the training images to visually inspect they have been correctly loaded
fig, ax = plt.subplots(5, 5)
k = 0
 
for i in range(5):
    for j in range(5):
        ax[i][j].imshow(X_train[k], aspect='auto')
        k += 1
 
plt.show()

# Model Fitting 

In [ ]:


# Defining the number of classes
K = len(set(y_train))

# Calculating total number of classes for the output layer
print("Number of classes:", K)

# Building the model using the functional API a common technique
i = Input(shape=X_train[0].shape)

# Convolutional layers with Batch Normalization and Dropout
x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.3)(x) 

x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.3)(x)  

x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.3)(x)  

x = Flatten()(x)
x = Dropout(0.4)(x)  

x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)  # Reduced number of neurons
x = Dropout(0.4)(x)  

# Output layer
x = Dense(K, activation='softmax')(x)

model = Model(i, x)

# Model description
model.summary()

# Compiling the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True)

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Data augmentation
batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

train_generator = data_generator.flow(X_train, y_train, batch_size)
steps_per_epoch = X_train.shape[0] // batch_size

# Training the model with data augmentation, early stopping, and learning rate scheduler
history = model.fit(train_generator, validation_data=(X_val, y_val),
                    steps_per_epoch=steps_per_epoch, epochs=100,
                    callbacks=[early_stopping, lr_scheduler])


# Evaluating model

# Visualization of model performance

In [ ]:
# plotting the accuracy of our model to visually see how well it's performing
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')


In [ ]:
# model performance on the training set
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
#priting training accuracy
print(f'Training Accuracy: {train_accuracy:.4f}')

# model performance on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
#priting validation accuracy
print(f'Validation Accuracy: {val_accuracy:.4f}')

In [ ]:
# Predicting the labels for the validation set since we don't have the labels for the test set 
y_pred_prob = model.predict(X_val)
y_pred_classes = np.argmax(y_pred_prob, axis=1)

# Calculating the confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred_classes)

# Calculating precision, recall, F1-score, and accuracy
precision = precision_score(y_val, y_pred_classes, average='weighted')
recall = recall_score(y_val, y_pred_classes, average='weighted')
f1 = f1_score(y_val, y_pred_classes, average='weighted')
accuracy = accuracy_score(y_val, y_pred_classes)

# Printing the metrics
print(f'Overall Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

# Printing a classification report as there's a handy function for this
print('\nClassification Report:')
print(classification_report(y_val, y_pred_classes))

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


### Interpreting Metrics
The overall accuracy represents the ratio of correctly predicted labels to the number of total labels.
My model correctly predicts 82.28% of the validation data.

### Precision: The ratio of true positive predictions and total predicted positives.
A precision of 82.73% means that when my model predicts a specific class it predicts correctly 82.73% of the time.

### Recall: This is the ratio of true positive predictions and total actual positives.
A recall of 82.28% shows that my model correctly identifies 82.28% of all actual instances of a class.

### F1-score.
An F1 score where it stands at 82.01% implies that there exists good balance between precision and recall in my model.

# Interpretation of the confusion matrix:
Each row of the matrix show the observations in the actual class, while each column represents the observations in a predicted class.
The diagonal elements indicate the number of observations that have been correctly classified, while off-diagonal elements are misclassifications.
The confusion matrix provides a detailed breakdown of correct and incorrect predictions for each class, allowing us to identify specific areas where the model may struggle.
We can see the column for class 7 has the most misclassifications.
Class 0 being misclassified as 5 vice versa.

# Submitting Predictions

In [ ]:
# Predict the labels for the test set to be submitted
test_predictions = model.predict(test_images)
test_labels = np.argmax(test_predictions, axis=1)

# Prepare the submission file
submission_df = pd.DataFrame({'id': test_ids, 'label': test_labels})
submission_df.to_csv('submission.csv', index=False)

# Summary

This analysis was carried out on images. Hence i have decided to use a Convolutional Neural Network (CNN) with the “relu” activation function. To make this model, I used functional API which is a very commonly used technique in deep learning.

I began by defining the input shape depending on the shapes of training images. Then, I added several convolutional layers, each followed by batch normalization and dropout layers to improve regularization. I specifically used kernel regularizer with L2 penalty to avoid overfitting and stabilize the training process using batch normalization.

For my convolutional layers, I started with 32 filters and then increased them to 64 filters in the next layers. Each set of convolutional layers was followed by a max pooling layer that reduces spatial dimensions and dropout layer that further prevents overfitting. As the network got deeper, I gradually increased the dropout rate for more regularization.

After convolutional layers, I flattened out the output and added a dense layer having 512 neurons along with dropout rate of 0.4. Output layer had number of neurons equal to classes’ number and had “softmax” activation function at final stage in order to get final classification probabilities.

In compiling my model, I utilized an optimizer called “adam”, which is well-suited for training deep neural networks and "sparse_categorical_crossentropy" being the loss function as the labels are integers that represent the classes. 

To further reduce overfitting i utilized earlystopping. This stops the training process when the validation loss does not show an improvement. I also used a learning rate scheduler which makes the learning rate lower over time to ensure the model is not simply learning the training data too well. 
To generalize the model i have used data augmentation techniques such as width and height shifts and horizontal flips. This helped increase the training data size. The batch size was set to 32 and data augmentation used on each batch. I set the number of epochs to be 100 to allow enough for the model to train better given all the steps i have taken to reduce overfitting. However if no improvements were seen in the validation loss the training would end earlier due to the implementation of early stopping.

An accuracy of 84.84% was achieved by the model during training.
The early stopping and learning rate scheduler introduced helped the model minimize overfitting. As a result, it has been able to generalize well on unseen data.
As we can see the precision, recall, and F1-score measures all confirm that the model gives equal consideration to different classes.
For example, some classes are well predicted by the model while others need improvement as seen from the confusion matrix.
On validation set performance was strong with an overall accuracy of 82.28%, precision of 82.73%, recall of 82.28% and F1-score of 82.01%.


# Possible Next Steps to Improve the Model:
1. Experimenting with different hyperparameters such as learning rate, batch size, and the number of layers/neurons to find the optimal configuration. 
    - Could also use Gridsearch/ Bayesian Optimization/ Random Search to help with this.
    
2. Trying more aggressive data augmentation techniques like zoom, and shear transformations to make the model more robust.
3. Adding more regularization techniques such as L1 regularization or increase the strength of L2 regularization to reduce overfitting.
5. Utilizing pre-trained models to leverage existing knowledge and improve performance.

